In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import BertTokenizer

In [2]:
# 1. Load dataset
dataset = load_dataset("imdb")

In [3]:
# 2. Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 3. Encode texts with tokenizer
def encode_with_bert(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding='max_length',
        max_length=128,
    )
    return {
        "input_ids": tokens["input_ids"],
        "attention_mask": tokens["attention_mask"],
        "label": example["label"]
    }

encoded_dataset = dataset.map(encode_with_bert, batched=True)
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


In [4]:
# 4. Prepare DataLoader
train_loader = DataLoader(encoded_dataset["train"], batch_size=32, shuffle=True)
test_loader = DataLoader(encoded_dataset["test"], batch_size=32)

# 5. Define LSTM-based model (without LoRA)
class StandardLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.i2h = nn.Linear(input_size, 4 * hidden_size)
        self.h2h = nn.Linear(hidden_size, 4 * hidden_size)
        self.hidden_size = hidden_size

    def forward(self, x, hidden):
        h, c = hidden
        gates = self.i2h(x) + self.h2h(h)
        i, f, g, o = gates.chunk(4, 1)
        i, f, g, o = torch.sigmoid(i), torch.sigmoid(f), torch.tanh(g), torch.sigmoid(o)
        c_next = f * c + i * g
        h_next = o * torch.tanh(c_next)
        return h_next, c_next


class TextLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm_cell = StandardLSTMCell(embed_dim, hidden_dim)
        self.hidden_dim = hidden_dim
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids)
        batch_size, seq_len, _ = embedded.size()
        h_t = torch.zeros(batch_size, self.hidden_dim, device=embedded.device)
        c_t = torch.zeros(batch_size, self.hidden_dim, device=embedded.device)

        for t in range(seq_len):
            h_t, c_t = self.lstm_cell(embedded[:, t, :], (h_t, c_t))

        return self.fc(h_t)


In [5]:
# 6. Initialize model
model = TextLSTM(
    vocab_size=tokenizer.vocab_size,
    embed_dim=128,
    hidden_dim=128,
    num_classes=2
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 7. Define optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()


In [6]:
# 8. Train model
for epoch in range(20):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")
torch.save(model.state_dict(), "lstm_imdb.pth")

Epoch 1, Loss: 0.6914
Epoch 2, Loss: 0.6653
Epoch 3, Loss: 0.4977
Epoch 4, Loss: 0.3271
Epoch 5, Loss: 0.2341
Epoch 6, Loss: 0.1571
Epoch 7, Loss: 0.0940
Epoch 8, Loss: 0.0561
Epoch 9, Loss: 0.0356
Epoch 10, Loss: 0.0254
Epoch 11, Loss: 0.0221
Epoch 12, Loss: 0.0167
Epoch 13, Loss: 0.0188
Epoch 14, Loss: 0.0093
Epoch 15, Loss: 0.0101
Epoch 16, Loss: 0.0102
Epoch 17, Loss: 0.0079
Epoch 18, Loss: 0.0072
Epoch 19, Loss: 0.0126
Epoch 20, Loss: 0.0060


In [ ]:
# 9. Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids)
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {correct / total:.2%}")



Test Accuracy: 77.83%
